In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/newsqa_train.json ./

In [ ]:
!pip install -U deep_translator
# source = source[:151] + " [an_s] " + source[151:179] + " [an_e] " + source[179:]

     |████████████████████████████████| 97 kB 3.9 MB/s 
     |████████████████████████████████| 97 kB 7.1 MB/s 
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.0.3 which is incompatible.


In [ ]:
import json
data_json = json.load(open('/content/newsqa_train.json', 'r'))

In [ ]:
len(data_json['data'])//8
# type(data_json['data'][0]['paragraphs'])

11568

In [ ]:
from deep_translator import GoogleTranslator
# translated = GoogleTranslator(source='auto', target='vi').translate(source)

In [ ]:
from tqdm import tqdm

In [ ]:
ws_data = {
    "version": '1.1',
    'data': []
}

In [ ]:
# k=0
# for data in data_json['data']:
#   for paragraph in data['paragraphs']:
#     for q in paragraph['qas']:
#       if q['is_impossible'] is not True:
#         k = k+1
# print(k)

In [ ]:
k=0
for data in tqdm(data_json['data'][11568:23136]):
  obj_1 = {
      "title": "",
      "paragraphs": []
  }
  for paragraph in data['paragraphs']:
    # context_0 = " ".join([" ".join(senc) for senc in annotator.tokenize(paragraph['context'])])
    context=paragraph['context'] 
    obj_2 = {
        'context' : "",
        'qas' : []
    }
    for qa in paragraph['qas']:
      obj_3 = {
          'question': GoogleTranslator(source='auto', target='vi').translate(qa['question']),
          'answers': [],
          'id': qa['id']
      }
      for answer in qa['answers']:
        new_context = context[0:answer['answer_start']] + "[an_s] " + context[answer['answer_start']:answer['answer_start']+len(answer['text'])] + " [an_e]" + context[answer['answer_start']+len(answer['text']):]
        try:
          new_context = GoogleTranslator(source='auto', target='vi').translate(new_context)
        except:
          k=k+1
          # print("error")
          continue
        try:
          answer_start= new_context.index("[an_s]")
          context = new_context.replace('[an_s] ', "").replace(' [an_e]','')
          text = context[answer_start:new_context.index("[an_e]")-8]
        except:
          # print("error")
          k =k +1
          continue
        obj_4 = {
            'answer_start':answer_start,
            'text': text
        }
        obj_3['answers'].append(obj_4)
      if len(obj_3['answers']) == 0:
        continue
      obj_2['qas'].append(obj_3)
      obj_2['context'] = context
    if len(obj_2['qas']) == 0:
      continue
    obj_1['paragraphs'].append(obj_2)
  if len(obj_1['paragraphs'])==0:
    continue
  ws_data['data'].append(obj_1)

print(k)

  0%|          | 66/92549 [01:00<23:38:56,  1.09it/s]


KeyboardInterrupt: ignored

In [ ]:
with open('/content/train_newqa_1.json', 'w') as outfile:
    json.dump(ws_data, outfile)

In [ ]:
!cp /content/train_newqa_1.json /content/drive/MyDrive